<a href="https://colab.research.google.com/github/ychzr9/RSA-and-ElGamal-Implementation/blob/main/rsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RSA algorithm is a public key encryption technique. It was invented by Rivest, Shamir and Adleman in year 1978.

RSA involves a public key and private key. The public key can be known to everyone- it is used to encrypt messages. Messages encrypted using the public key can only be decrypted with the private key. The private key needs to be kept secret. Calculating the private key from the public key is very difficult.

RSA works in the following way: 

1.   Generating keys:


*   Choose two different random prime numbers *p* and *q*. This should be kept secret.
*   Calculate *n=pq*
*   Calculate 	$\phi$*(n)=(p-1)(q-1)*
*   Choose an integer *e* such that *1< e < $\phi(n)$*, and *e* is co-prime to *$\phi$(n)*. 
  ( *e* is released as the public key exponent.)
*   Compute *d* to satisfy the congruence relation *de $\equiv$ 1 (mod $\phi$(n))*.
   (*d* is kept as the private key exponent)

2.   Encrypting message: 
 
  Alice gives her public key *(n,e)* to Bob and keeps her private key secret. Bob wants to send message *M* to Alice.

  The operation used is:   
  *$c=m^e$ mod(n)*.

3. Decrypying message: 

  Alice can recover *m* from *c*, by using her private key *d*. 
  
  The basic operation is the following :

  *m $\equiv$ $c^d$ (mod n)*.

Let us see the process in python code: 





In [ ]:
# Giving the main function 
def main():
   
  selection = input('Choose: Encryption // Decryption (Press 1 to encrypt, Press 2 to decrypt): ')
  if (selection == '1'):
      message = input('Give the message that will be encrypted: \n')
      print(encryption(message))

  elif (selection == '2'):
      message = input('Give the message that will be decrypted: \n')
      print(decryption(message))
  else:
      print('Error')

main()

In [ ]:
# First, giving the function that returns gratest common divisor of two integers a and b;
def gcd(a, b):
    if a == 0 :
        return b
     
    return gcd(b%a, a)
 

In [ ]:
# Giving the function that calculates the Extended Euclidean Algorithm
def ext_eucld_alg(a, b):
  x,y, u,v = 0,1, 1,0
  while a != 0:
    q, r = b//a, b%a
    m, n = x-u*q, y-v*q
    b,a, x,y, u,v = a,r, u,v, m,n
  gcd = b
# Returning greatest common divisor of a and b, coefficient of a, coefficient of b  
  return gcd, x, y

      

In [ ]:
# Generate two prime numbers between 100 and 500.
import random
from sympy import *
primes = [i for i in range(100,500) if isprime(i)]
p = random.choice(primes)
q = random.choice(primes)


# Now, define n, phi_n;
n=p*q 
phi_n=(p-1)*(q-1)


In [ ]:
# Choosing the public key e, that is between (2,phi_n) and prime with phi_n
def choose_e(phi_n):
  while (True):
    e = random.randrange(2, phi_n)
    if (gcd(e, phi_n) == 1):
        return e
e= choose_e(phi_n)
#print(e)

'''
 After choosing 'e', now we calculate private key 'd' using Extended Euclidean Algorithm  
 Note that d is an integer in the interval (1, phi_n) and ed=1 (mod phi_n)
'''
gcd, x, y = ext_eucld_alg(choose_e(phi_n), phi_n)
if (x < 0):
  d = x + phi_n
else:
  d = x

#print(d)

In [ ]:
# Write parameters into a file 
f_parameters_public = open('parametersenc.txt', 'w')
f_parameters_public.write(str(n)+'\n')
f_parameters_public.write(str(e)+'\n')
f_parameters_public.close()
f_parameters_private = open('parametersdec.txt','w')
f_parameters_private.write(str(n)+'\n')
f_parameters_private.write(str(d)+'\n')
f_parameters_private.close()

Now, we selected random primes 'p' and 'q', public key 'e' and calculated private key 'd'. The following is the encryption part:

In [ ]:
def encryption(plaintxt, file='parametersenc.txt', blocksize=2):
  # First read parameters 'n' and 'e' from the file
  f = open(file,'r')
  e = int(f.readline())
  n = int(f.readline())
  f.close()

  encryptedblock = []

# Initilization of the ciphertext with the first character of plaintext:
# Using ord function, give ascii value of first character of plaintext as initilization 
  if(len(plaintxt)>0):
    ciphertxt = ord(plaintxt[0])

  for j in range(1, len(plaintxt)):
    if(j%blocksize == 0):     
      # in the case of block size is at the highest, append encrypted ciphertext to the list
      encryptedblock.append(ciphertxt)
      # multiply by 1000 to shift the digits over to the left by 3 places
      ciphertxt = ciphertxt * 1000 + ord(message[i])
  encryptedblock.append(ciphertxt)

# Finally, doing the encryption operation according to the length of encryptedblock 
  for j in range(len(encryptedblock)):
    encryptedblock[j] = str((encryptedblock[j]**e) % n) # (plaintext)^e (mod n) 

  final_ciphertxt=" ".join(encryptedblock) # add encrypted blocks together 

  return final_ciphertxt

In [ ]:
def decryption(block, blocksize = 2):
  # First, read the private keys 'n' & 'd' from the file 
  f = open('parametersdec.txt', 'r')
  d = int(f.readline())
  n = int(f.readline())
  f.close()

  # turning the string into a list of integers
  blockslist = block.split(' ')
  integerlistblock = []

  for i in blockslist:
    integerlistblock.append(int(i))

  message = ""
 
  # In the integer list, convert each integer to the characters
  for i in range(len(integerlistblock)):
    integerlistblock[i] = (integerlistblock[i]**d) % n      # decryption operation: (integer)^d (modn), for each integer in the list
        
    k = ""
    # take apart each block into its ASCII codes for each character
    # and store it in the message string
    for j in range(blocksize):
        k = chr(integerlistblock[i] % 1000) + k
        integerlistblock[i] //= 1000
    message += k

  return message


